# MCMC3.5: Jackknife Resampling

To estimate the error of observables accurately, it is recommended to use the Jackknife resampling method.

## Binning

Previously, we faced a large errorbar problem at low temperature in this program:

In [1]:
using ResumableFunctions
using SparseArrays
using LinearAlgebra
const Jx = 1 / 3 # oppposite sign to Motome's
const Jy = 1 / 3
const Jz = 1 / 3
function Metropolis(βF::Float64, βFnew::Float64)::Bool
    βF - βFnew > log(rand())
end
function openhoneycomb(Lx::Int64, Ly::Int64)::Tuple
    N = 2Lx * Ly
    nnx = zip(1 : 2 : (N - 1), 2 : 2 : N)
    nny = Iterators.flatten(zip((1 + 2i) : 2Lx : (2Lx * (Ly - 1)  + 1 + 2i), 2i : 2Lx : (2Lx * (Ly - 1)  + 2i)) for i in 1 : (Lx - 1))
    nnz = zip(1 : 2 : (N - 1), Iterators.flatten(((2Lx + 2) : 2 : N, 2 : 2 : 2Lx)))
    plaquette = Iterators.flatten(zip((Lx * (i - 1) + 1) : (Lx * (i - 1) + Lx - 1), (Lx * (i - 1) + 2) : (Lx * (i - 1) + Lx)) for i in 1 : Ly)
    N, nnx, nny, nnz, plaquette
end
@resumable function measurementflux(method::Function, lattice::Function, β::Float64, Lx::Int64, Ly::Int64)::Float64
    N, nnx, nny, nnz, plaquette = lattice(Lx, Ly)
    iter = Iterators.flatten((Iterators.product(J, nn) for (J, nn) in [(Jx, nnx), (Jy, nny), (Jz, nnz)]))
    h = spzeros(Complex{Float64}, N, N)
    for (J, nn) in iter
        h[nn[1], nn[2]] = 0.5im * J
        h[nn[2], nn[1]] = -0.5im * J
    end
    NNz = collect(nnz)
    Nz = length(NNz)
    η = ones(Int64, Nz)
    βF = 0.0
    hdense = Array(h)
    plaq = collect(plaquette)
    Np = length(plaq)
    while true
        for i in 1 : Nz
            j = rand(1 : Nz)
            hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
            hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            ev = eigvals(Hermitian(hdense))
            positiveev = Iterators.drop(ev, N >> 1)
            βFnew = -sum((log(2.0 * cosh(β * ϵ / 2.0)) for ϵ in positiveev))
            if method(βF, βFnew)
                η[j] = -η[j]
                βF = βFnew
            else
                hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
                hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            end
        end
        @yield sum((η[k] * η[l] for (k, l) in plaq)) / Np
    end
end

measurementflux (generic function with 1 method)

One reason is the discreteness of the returned value "flux."

In [2]:
mcstep = Iterators.drop(measurementflux(Metropolis, openhoneycomb, 100.0, 4, 4), 10000)
foreach(println, Iterators.take(mcstep, 10))

0.6666666666666666
0.0
0.3333333333333333
0.0
0.6666666666666666
0.5
0.3333333333333333
0.3333333333333333
0.0
0.6666666666666666


We have to flatten these quantized values by binning.

In [3]:
using Statistics
Nsample = 10000
Nbin = 10
Nbinsize = Nsample ÷ Nbin
iter = Iterators.partition(Iterators.take(mcstep, Nsample), Nbinsize)
bin = collect(map(mean, iter))

10-element Array{Float64,1}:
 0.29233333333333333
 0.3006666666666667 
 0.2909999999999999 
 0.3093333333333332 
 0.2899999999999999 
 0.31816666666666665
 0.2918333333333334 
 0.3105             
 0.30783333333333335
 0.3006666666666667 

Now it works!

In [4]:
m = mean(bin)
s = stdm(bin, m) / sqrt(length(bin))
println("$m ± $s")

0.30123333333333335 ± 0.003128542436627282


Nbinsize has to be determined based on the autocorrelation. In order to reduce Nbin to get a more acculate result at low temperature, we need to implement some global updating algorithm.

## Delete-1 jackknife resampling

Delete-1 jackknife resampling is simply implemented in https://github.com/ararslan/Jackknife.jl. However, the function is limited, so I will newly define functions for the jackknife resampling.

In [5]:
function leaveoneout(before::Function, after::Function, v::AbstractVector)
    ind = eachindex(v)
    map(i -> after(mean(map(before, view(v, filter(!isequal(i), ind))))), ind)
end
meanJ(b::Function, a::Function, v::AbstractVector) = mean(leaveoneout(b, a, v))
stdmJ(b::Function, a::Function, v::AbstractVector, m) = stdm(leaveoneout(b, a, v), m, corrected = false) * sqrt(length(v) - 1)
stdJ(b::Function, a::Function, v::AbstractVector) = stdmJ(b, a, v, meanJ(b, a, v))

stdJ (generic function with 1 method)

The functions are based on Statistics.jl and Jackknife.jl, so please see their reference to know how it works. I again use measurementEf for the demonstration.

In [6]:
@resumable function measurementEf(method::Function, lattice::Function, β::Float64, Lx::Int64, Ly::Int64)::Vector{Float64}
    N, nnx, nny, nnz, plaquette = lattice(Lx, Ly)
    iter = Iterators.flatten((Iterators.product(J, nn) for (J, nn) in [(Jx, nnx), (Jy, nny), (Jz, nnz)]))
    h = spzeros(Complex{Float64}, N, N)
    for (J, nn) in iter
        h[nn[1], nn[2]] = 0.5im * J
        h[nn[2], nn[1]] = -0.5im * J
    end
    NNz = collect(nnz)
    Nz = length(NNz)
    η = ones(Int64, Nz)
    βF = 0.0
    β₂ = β * 0.5
    hdense = Array(h)
    ev = zeros(Float64, N)
    while true
        for i in 1 : Nz
            j = rand(1 : Nz)
            hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
            hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            evnew = eigvals(Hermitian(hdense))
            βFnew = -sum(@. log(exp(β₂ * evnew[(N >> 1 + 1) : end]) + exp(-β₂ * evnew[(N >> 1 + 1) : end])))
            if method(βF, βFnew)
                η[j] = -η[j]
                βF = βFnew
                ev .= evnew
            else
                hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
                hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            end
        end
        Ef = -sum(@. ev[(N >> 1 + 1) : end] * tanh(β₂ * ev[(N >> 1 + 1) : end] )) * 0.5
        ∂Ef∂β = -sum(@. (ev[(N >> 1 + 1) : end] ^ 2) * (sech(β₂ * ev[(N >> 1 + 1) : end]) ^ 2)) * 0.25
        @yield [Ef, ∂Ef∂β]
    end
end

measurementEf (generic function with 1 method)

It is ok to first assume the bin size to be 1.

In [7]:
const β = 10.0
mcstep2 = Iterators.drop(measurementEf(Metropolis, openhoneycomb, β, 4, 4), 10000)
iter2 = Iterators.take(mcstep2, Nsample)
data = collect(iter2)

10000-element Array{Array{Float64,1},1}:
 [-1.67029, -0.0442268]
 [-1.69011, -0.0475176]
 [-1.67796, -0.046018] 
 [-1.69761, -0.0492358]
 [-1.69617, -0.0486713]
 [-1.68042, -0.0454945]
 [-1.68038, -0.0462557]
 [-1.6869, -0.0469995] 
 [-1.68206, -0.045995] 
 [-1.67791, -0.0454503]
 [-1.67757, -0.0450611]
 [-1.6701, -0.044125]  
 [-1.6782, -0.0455557] 
 ⋮                     
 [-1.68438, -0.0468955]
 [-1.68829, -0.0475562]
 [-1.67295, -0.0443593]
 [-1.68162, -0.0458104]
 [-1.67568, -0.0455054]
 [-1.68206, -0.045995] 
 [-1.69032, -0.047401] 
 [-1.6908, -0.0476395] 
 [-1.67876, -0.0455681]
 [-1.67555, -0.0454791]
 [-1.6795, -0.0458803] 
 [-1.67911, -0.0457078]

By setting after = before = identity, meanJ and stdmJ work in the same way as mean and stdm, respectively.

In [8]:
m2 = meanJ(identity, identity, data)
s2 = stdmJ(identity, identity, data, m2)
println("Ef = $(m2[1]) ± $(s2[1]), ∂Ef∂β = $(m2[2]) ± $(s2[2])")

Ef = -1.6834300533402857 ± 7.513021272429419e-5, ∂Ef∂β = -0.04647179598647452 ± 1.3320629221053692e-5


This agrees with the standard estimation method for the error bars.

In [9]:
std(data) / sqrt(length(data))

2-element Array{Float64,1}:
 7.513021272452648e-5 
 1.3320629221066421e-5

For such mean values (i.e. op = mean), the jackknife resampling is apparently overkill. However, to estimate the error for the values like the specific heat, the jackknife resampling is very effective.

In [10]:
TTCv(v::Vector{Float64}) = [v[1] ^ 2 - v[2], v[1]]
Cv(meanTTCv::Vector{Float64}) = (β ^ 2) * (meanTTCv[1] - meanTTCv[2] ^ 2)
m3 = meanJ(TTCv, Cv, data)
s3 = stdmJ(TTCv, Cv, data, m3)
println("Cv = $m3 ± $s3")

Cv = 4.652823583000198 ± 0.0013381528065419326


## Autocorrelation

The simplest way to estimate autocorrelation is by changing the size of binning and estimating its errors by jackknife resampling.

In [11]:
binning = Iterators.partition(data, 2)
bin2 = collect(map(mean, binning))

5000-element Array{Array{Float64,1},1}:
 [-1.6802, -0.0458722] 
 [-1.68778, -0.0476269]
 [-1.68829, -0.0470829]
 [-1.68364, -0.0466276]
 [-1.67998, -0.0457227]
 [-1.67383, -0.0445931]
 [-1.68177, -0.0463361]
 [-1.68527, -0.0466862]
 [-1.68741, -0.0471112]
 [-1.68715, -0.0470853]
 [-1.68352, -0.0464573]
 [-1.68451, -0.0465759]
 [-1.69309, -0.04798]  
 ⋮                     
 [-1.6781, -0.0457569] 
 [-1.68814, -0.0473952]
 [-1.68142, -0.0461936]
 [-1.67962, -0.0456494]
 [-1.69026, -0.0473906]
 [-1.68342, -0.0465292]
 [-1.68633, -0.0472258]
 [-1.67729, -0.0450849]
 [-1.67887, -0.0457502]
 [-1.69056, -0.0475202]
 [-1.67715, -0.0455236]
 [-1.67931, -0.045794] 

In [12]:
m4 = meanJ(TTCv, Cv, bin2)
s4 = stdmJ(TTCv, Cv, bin2, m4)
println("Cv = $m4 ± $s4")

Cv = 4.650074637685684 ± 0.0013552176999298508


The fact that the binsize does not affect the expectation value (or the errorbar) too much means that the autocorrelation length is about 1 step. Note that at low temperature the binsize strongly affects the expectation value, which means that the binsize must be taken to be large enough. Here I estimate the autocorrelation length by another method.

In [13]:
using StatsBase
StatsBase.autocor(map(x -> x[1], data))

41-element Array{Float64,1}:
  1.0                  
  0.032346373132680156 
 -0.006038199529635438 
  0.0006474151419614435
  0.003944549058417232 
 -0.004609663637028737 
  0.006022605545539759 
  0.013950230571159065 
 -0.013518612610622932 
  0.004534993483739318 
 -0.007802817448246155 
 -0.001625183775702386 
  0.0014292621092096285
  ⋮                    
  0.004988034511773137 
  0.0016836658911762892
 -0.002934118662804246 
 -0.019698747677354307 
  0.003012520513069854 
 -0.01100987427370909  
 -0.018772591824874958 
 -0.0009132537998946528
  0.0041570900455745454
  0.0019064356793108783
 -0.005846369301569821 
 -0.008753570672815075 

Rapid decay in the autocorrelation function means that the autocorrelation length is less than 1.

## Bootstrap method

~ under construction ~

**Exercise**: implement a Bootstrap method.